In [1]:
import os
import sys
import numpy as np
import lasagne as L

sys.path.append('../')
sys.path.append('../scripts/')

from os.path import join
from squad_tools import load_glove
from AnswerBot import AnswerBot
from QANet import QANet
from my_tokenize import tokenize

Using cuDNN version 5105 on context None
Preallocating 3661/12204 Mb (0.300000) on cuda

floatX == float32
device == cuda



Mapped name None to device cuda: GeForce GTX TITAN X (0000:01:00.0)


In [2]:
glove_path = '/pio/data/data/glove_vec/6B/glove.6B.300d.txt'
squad_base_path = '/pio/data/data/squad/'

squad_path = join(squad_base_path, 'preproc')
glove_words, glove_embs = load_glove(glove_path)
w_to_i = {w:i for (i,w) in enumerate(glove_words)}
voc_size = glove_embs.shape[0]
NAW_token = glove_words.index('<not_a_word>')

# Demo

In [4]:
abot = AnswerBot(
    '../models/best/model.ep09.npz',
    glove_embs,
    glove_words,
    train_unk=True,
    negative=False,
    conv='valid')

abot_neg = AnswerBot(
    '../models/model_neg03/model.ep07.npz',
    glove_embs,
    glove_words,
    train_unk=True,
    negative=True,
    conv='full')

Building the model...
Using dropout after wiq calculation.
Compiling theano functions:
    get_intermediate_results_fn...
    get_start_probs_fn...
    get_end_probs_fn...
Done
Building the model...
Using dropout after wiq calculation.
Compiling theano functions:
    get_intermediate_results_fn...
    get_start_probs_fn...
    get_end_probs_fn...
Done


In [6]:
# x is a fragment in natural language, qs are questions
# examples of usage are shown below

x = u"The torch relay started on 24 October 2017 in Greece and ended at "\
     "the start of the Olympics on 9 February 2018. On 1 November 2017 the relay "\
     "entered Korea. The relay lasted 101 days. There were 7,500 torch bearers to "\
     "represent the Korean population of 75 million people. There were also 2018 support "\
     "runners to guard the torch and act as messengers."
    
q = u"How long was the torch relay?"
abot.get_answers([q], [x])

[([u'101', u'days'], 0.93698061)]

In [72]:
x1 = u"The Battle of Grunwald, First Battle of Tannenberg or Battle of Žalgiris, was fought on 15 July 1410 during the Polish–Lithuanian–Teutonic War. The alliance of the Kingdom of Poland and the Grand Duchy of Lithuania, led respectively by King Władysław II Jagiełło (Jogaila) and Grand Duke Vytautas, decisively defeated the German–Prussian Teutonic Knights, led by Grand Master Ulrich von Jungingen."
x2 = u'Frédéric François Chopin (1 March 1810 – 17 October 1849) was a Polish composer and virtuoso pianist of the Romantic era who wrote primarily for the solo piano. He gained and has maintained renown worldwide as a leading musician of his era, whose "poetic genius was based on a professional technique that was without equal in his generation."'

q = u"When was Battle of Grunwald?"
print abot.get_answers([q,q], [x1,x2])
print abot_neg.get_answers([q,q], [x1,x2])

[([u'15', u'july', u'1410'], 0.94606912), ([u'1', u'march', u'1810'], 0.70233828)]
[([u'15', u'july', u'1410'], 0.53974926), (['<not_a_word>'], 0.959436)]


### Samples from SQuAD dev

In [6]:
import json

with open(join(squad_path, 'dev.json')) as f:
    dev = json.load(f)

In [42]:
start_index = 999
num = 5

x = ' '.join(dev[start_index][2])
qs = [' '.join(d[1]) for d in dev[start_index:start_index+num]]
anss = [d[0] for d in dev[start_index:start_index+num]]
anss = [[a[1] for a in d] for d in anss]
print x, '\n'
for i in range(len(qs)):
    print 'Q:', qs[i]
    print 'A:', anss[i]
    print
abot.get_answers(qs, [x]*len(qs))

other green spaces in the city include the botanic garden and the university library garden . they have extensive botanical collection of rare domestic and foreign plants , while a palm house in the new orangery displays plants of subtropics from all over the world . besides , within the city borders , there are also : pole mokotowskie ( a big park in the northern mokotów , where was the first horse racetrack and then the airport ) , park ujazdowski ( close to the sejm and john lennon street ) , park of culture and rest in powsin , by the southern city border , park skaryszewski by the right vistula bank , in praga . the oldest park in praga , the praga park , was established in 1865 – 1871 and designed by jan dobrowolski . in 1927 a zoological garden ( ogród zoologiczny ) was established on the park grounds , and in 1952 a bear run , still open today . 

Q: what type of space in warsaw are the botanic garden and university library garden ?
A: [u'green', u'green', u'green']

Q: where i

[([u'green'], 0.81224889),
 ([u'new', u'orangery'], 0.34523931),
 ([u'pole', u'mokotowskie'], 0.34411412),
 ([u'park', u'ujazdowski'], 0.95149535),
 ([u'1865', u'\u2013', u'1871'], 0.77209204)]

In [43]:
start_index = 4321
num = 5

x = ' '.join(dev[start_index][2])
qs = [' '.join(d[1]) for d in dev[start_index:start_index+num]]
anss = [d[0] for d in dev[start_index:start_index+num]]
anss = [[a[1] for a in d] for d in anss]
print x, '\n'
for i in range(len(qs)):
    print 'Q:', qs[i]
    print 'A:', anss[i]
    print
abot.get_answers(qs, [x]*len(qs))

following the cretaceous – paleogene extinction event , the extinction of the dinosaurs and the wetter climate may have allowed the tropical rainforest to spread out across the continent . from 66 – 34 mya , the rainforest extended as far south as 45 ° . climate fluctuations during the last 34 million years have allowed savanna regions to expand into the tropics . during the oligocene , for example , the rainforest spanned a relatively narrow band . it expanded again during the middle miocene , then retracted to a mostly inland formation at the last glacial maximum . however , the rainforest still managed to thrive during these glacial periods , allowing for the survival and evolution of a broad diversity of species . 

Q: which type of climate may have allowed the rainforest to spread across the continent ?
A: [u'the wetter climate may have allowed the tropical rainforest to spread out across the continent.', u'wetter', u'wetter']

Q: what has allowed for the savanna region to expand 

[([u'tropical'], 0.51566482),
 ([u'climate', u'fluctuations'], 0.97780937),
 ([u'the', u'oligocene', u','], 0.19961046),
 ([u'middle', u'miocene'], 0.492405),
 ([u'still', u'managed', u'to', u'thrive'], 0.31349531)]

In [61]:
start_index = 598
num = 6

x = ' '.join(dev[start_index][2])
qs = [' '.join(d[1]) for d in dev[start_index:start_index+num]]
anss = [d[0] for d in dev[start_index:start_index+num]]
anss = [[a[1] for a in d] for d in anss]
print x, '\n'
for i in range(len(qs)):
    print 'Q:', qs[i]
    print 'A:', anss[i]
    print
abot.get_answers(qs, [x]*len(qs))

quickbooks sponsored a " small business big game " contest , in which death wish coffee had a 30 - second commercial aired free of charge courtesy of quickbooks . death wish coffee beat out nine other contenders from across the united states for the free advertisement . 

Q: what company won a free advertisement due to the quickbooks contest ?
A: [u'death wish coffee', u'death wish coffee', u'death wish coffee']

Q: how long was the death wish coffee commercial ?
A: [u'30-second', u'30-second', u'30-second']

Q: besides death wish coffee , how many other competitors participated in the contest ?
A: [u'nine', u'nine', u'nine']

Q: which company won a contest to have their ad shown for free during super bowl 50 ?
A: [u'death wish coffee', u'death wish coffee', u'death wish coffee']

Q: how many other contestants did the company , that had their ad shown for free , beat out ?
A: [u'nine', u'nine', u'nine']

Q: which company sponsored a contest called " small business big game " ?
A: [u'qu

[([u'coffee'], 0.33887854),
 ([u'30', u'-', u'second'], 0.77624053),
 ([u'nine'], 0.62543964),
 ([u'quickbooks'], 0.29217994),
 ([u'nine'], 0.79637706),
 ([u'quickbooks'], 0.99977797)]

In [62]:
start_index = 200
num = 6

x = ' '.join(dev[start_index][2])
qs = [' '.join(d[1]) for d in dev[start_index:start_index+num]]
anss = [d[0] for d in dev[start_index:start_index+num]]
anss = [[a[1] for a in d] for d in anss]
print x, '\n'
for i in range(len(qs)):
    print 'Q:', qs[i]
    print 'A:', anss[i]
    print
abot.get_answers(qs, [x]*len(qs))

despite waiving longtime running back deangelo williams and losing top wide receiver kelvin benjamin to a torn acl in the preseason , the carolina panthers had their best regular season in franchise history , becoming the seventh team to win at least 15 regular season games since the league expanded to a 16 - game schedule in 1978 . carolina started the season 14 – 0 , not only setting franchise records for the best start and the longest single - season winning streak , but also posting the best start to a season by an nfc team in nfl history , breaking the 13 – 0 record previously shared with the 2009 new orleans saints and the 2011 green bay packers . with their nfc - best 15 – 1 regular season record , the panthers clinched home - field advantage throughout the nfc playoffs for the first time in franchise history . ten players were selected to the pro bowl ( the most in franchise history ) along with eight all - pro selections . 

Q: who had the best record in the nfc ?
A: [u'caroli

[([u'carolina', u'panthers'], 0.39416635),
 ([u'ten'], 0.73643702),
 ([u'eight'], 0.72282839),
 ([u'kelvin', u'benjamin'], 0.93562293),
 ([u'1978'], 0.99421757),
 ([u'2009'], 0.93824393)]

In [69]:
start_index = 5132
num = 5

x = ' '.join(dev[start_index][2])
qs = [' '.join(d[1]) for d in dev[start_index:start_index+num]]
anss = [d[0] for d in dev[start_index:start_index+num]]
anss = [[a[1] for a in d] for d in anss]
print x, '\n'
for i in range(len(qs)):
    print 'Q:', qs[i]
    print 'A:', anss[i]
    print
abot.get_answers(qs, [x]*len(qs))

sir charles lyell first published his famous book , principles of geology , in 1830 . this book , which influenced the thought of charles darwin , successfully promoted the doctrine of uniformitarianism . this theory states that slow geological processes have occurred throughout the earth ' s history and are still occurring today . in contrast , catastrophism is the theory that earth ' s features formed in single , catastrophic events and remained unchanged thereafter . though hutton believed in uniformitarianism , the idea was not widely accepted at the time . 

Q: first published by sir charles lyell in 1830 this book was called what ?
A: [u'principles of geology', u'principles of geology', u'principles of geology']

Q: what doctrine did the doctrine of the principles of geology successfully promote ?
A: [u'uniformitarianism', u'uniformitarianism', u'uniformitarianism']

Q: which theory states that slow geological processes are still occurring today , and have occurred throughout ear

[([u'principles', u'of', u'geology'], 0.96457863),
 ([u'the', u'doctrine', u'of', u'uniformitarianism'], 0.63317519),
 ([u'uniformitarianism'], 0.47548464),
 ([u'catastrophism'], 0.99940991),
 ([u'charles', u'darwin'], 0.6520983)]

### For negative examples

In [16]:
start_index = 7235
num = 3

x = ' '.join(dev[start_index][2])
qs = [' '.join(d[1]) for d in dev[start_index:start_index+num]]
anss = [d[0] for d in dev[start_index:start_index+num]]
anss = [[a[1] for a in d] for d in anss]
print x, '\n'
for i in range(len(qs)):
    print 'Q:', qs[i]
    print 'A:', anss[i]
    print
print abot.get_answers(qs, [x]*len(qs))
print abot_neg.get_answers(qs, [x]*len(qs))

negqs = [u'What is the population of Los Angeles?',
         u'What is the smallest city in Oklahoma?',
         u'Who founded Jacksonville?']
print 
for i in range(len(negqs)):
    print 'Q n:', negqs[i]
print
    
print abot.get_answers(negqs, [x]*len(negqs))
print abot_neg.get_answers(negqs, [x]*len(negqs))

jacksonville is the largest city by population in the u . s . state of florida , and the largest city by area in the contiguous united states . it is the county seat of duval county , with which the city government consolidated in 1968 . consolidation gave jacksonville its great size and placed most of its metropolitan population within the city limits ; with an estimated population of 853 , 382 in 2014 , it is the most populous city proper in florida and the southeast , and the 12th most populous in the united states . jacksonville is the principal city in the jacksonville metropolitan area , with a population of 1 , 345 , 596 in 2010 . 

Q: which florida city has the biggest population ?
A: [u'jacksonville', u'jacksonville', u'jacksonville']

Q: what was the population jacksonville city as of 2010 ?
A: [u'1,345,596', u'1,345,596', u'1,345,596']

Q: based on population alone , what is jacksonville ' s ranking in the united states ?
A: [u'12th', u'12th', u'12th']

[([u'jacksonville'], 

In [73]:
start_index = 2341
num = 3

x = ' '.join(dev[start_index][2])
qs = [' '.join(d[1]) for d in dev[start_index:start_index+num]]
anss = [d[0] for d in dev[start_index:start_index+num]]
anss = [[a[1] for a in d] for d in anss]
print x, '\n'
for i in range(len(qs)):
    print 'Q:', qs[i]
    print 'A:', anss[i]
    print
print abot.get_answers(qs, [x]*len(qs))
print abot_neg.get_answers(qs, [x]*len(qs))

negqs = [u'Who was Martin Luther?',
         u'How many epistles did St. Paul write?',
         u'When did Luther annouce his three treatises?']
print 
for i in range(len(negqs)):
    print 'Q n:', negqs[i]
print
    
print abot.get_answers(negqs, [x]*len(negqs))
print abot_neg.get_answers(negqs, [x]*len(negqs))

luther justified his opposition to the rebels on three grounds . first , in choosing violence over lawful submission to the secular government , they were ignoring christ ' s counsel to " render unto caesar the things that are caesar ' s " ; st . paul had written in his epistle to the romans 13 : 1 – 7 that all authorities are appointed by god and therefore should not be resisted . this reference from the bible forms the foundation for the doctrine known as the divine right of kings , or , in the german case , the divine right of the princes . second , the violent actions of rebelling , robbing , and plundering placed the peasants " outside the law of god and empire " , so they deserved " death in body and soul , if only as highwaymen and murderers . " lastly , luther charged the rebels with blasphemy for calling themselves " christian brethren " and committing their sinful acts under the banner of the gospel . 

Q: what were the protesters doing with christ ' s counsel ?
A: [u"ignorin

In [89]:
start_index = 5253
num = 3

x = ' '.join(dev[start_index][2])
qs = [' '.join(d[1]) for d in dev[start_index:start_index+num]]
anss = [d[0] for d in dev[start_index:start_index+num]]
anss = [[a[1] for a in d] for d in anss]
print x, '\n'
for i in range(len(qs)):
    print 'Q:', qs[i]
    print 'A:', anss[i]
    print
print abot.get_answers(qs, [x]*len(qs))
print abot_neg.get_answers(qs, [x]*len(qs))

negqs = [u'When was Widex founded?',
         u'How many UK citizens have hearing problems?',
         u'What is the average length of a motorway?']
print 
for i in range(len(negqs)):
    print 'Q n:', negqs[i]
print
    
print abot.get_answers(negqs, [x]*len(negqs))
print abot_neg.get_answers(negqs, [x]*len(negqs))

in a report , published in early february 2007 by the ear institute at the university college london , and widex , a danish hearing aid manufacturer , newcastle was named as the noisiest city in the whole of the uk , with an average level of 80 . 4 decibels . the report claimed that these noise levels would have a negative long - term impact on the health of the city ' s residents . the report was criticized , however , for attaching too much weight to readings at arbitrarily selected locations , which in newcastle ' s case included a motorway underpass without pedestrian access . 

Q: what ' s the average decibel level of noise in newcastle ?
A: [u'80.4', u'80.4', u'80.4 decibels']

Q: what type of impact can the residents of newcastle expect the city ' s noise to have on them ?
A: [u'negative', u'negative long-term', u'negative long-term impact on the health']

Q: what was one location the noise readings in newcastle were taken at ?
A: [u'a motorway underpass', u'a motorway underpass